# Homework 2
## Hosley, Brandon
## OPER 785

### Task:
Expand the example such that the network is capable of solving the XOR operator (remember: the XOR operator evaluates two binary inputs and returns a 0 when the inputs are equal and a 1 when the inputs are different). It should be sufficient to add an additional layer of TLUs to solve this problem, but there is the problem of 2 sets of weights to account for in your back propagation.

Deliverables: Well documented code and output (Jupyter Notebook is sufficient for both, but you may use any language, interpreter, or compiler). Important notations include beginning weights, final weights, and dimensionality of arrays.

## Example Artificial Neural Network (from Class)
Follows example from Polycode channel on YouTube.

In [1]:
# Import the appropriate packages
import numpy as np

Define a neural network class. The class will have a sigmoid activation function, its derivative, 
feedforward and backpropogation steps, and a single output.
 
As a single layer network, the network is incapable of calculating complex logical operators. Multiple
layers would be required for something as simple as a logical "XOR" function.

In [2]:
class NeuralNetwork:
    def __init__(self, x, y):
        self.x              = np.array(x)
        self.y              = np.array(y)
        
        # Set weights
        self.weights        = 2 * np.random.random((x.shape[1],y.shape[1])) - 1

        self.error          = np.zeros(self.y.shape)
        self.output         = np.zeros(self.y.shape)
        self.adjustments    = np.zeros(self.y.shape)

    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self,x):
        return x* (1-x)

    def train(self, i):
        for iteration in range (i):
            # Feedforward
            self.output = self.sigmoid(np.dot(self.x, self.weights))
    
            # Back propogation
            self.error = self.y - self.output
            self.adjustments = self.error * self.sigmoid_derivative(self.output)
            self.weights += np.dot(self.x.T, self.adjustments)

Once the class is defined, we can instantiate it through defining new variables. Let's consider a case of three
binary input variables with a single binary output. The data can be seen to represent (x1 "AND" x3).

In [3]:
x = np.array([[0, 0, 1],[1, 1, 1],[1, 0, 1],[0, 1, 1]])
y = np.array([[0],[1],[1],[0]])
print(x.shape, y.shape)

(4, 3) (4, 1)


Once we have the inputs and outputs, we can instantiate an instance of the NeuralNetwork object.

In [4]:
ann = NeuralNetwork(x,y)

We train the neural network for a specified number of epochs and show the outputs after training. 

In [5]:
ann.train(100)
    
print(ann.output)

[[0.10993431]
 [0.91093839]
 [0.92696043]
 [0.09053048]]


Note, the outputs from the trained network are "close to" the desired output (training values). More training epochs
would get us closer to the desired values [0, 1, 1, 0]. So let's try some additional training epochs (Note: since the
weights are not reset, we can just call the training function again).

In [6]:
ann.train(1400) #This will give us 1500 replications
print(ann.output)
print(ann.weights)

[[0.02568097]
 [0.97910164]
 [0.9830782 ]
 [0.02081371]]
[[ 7.6987835 ]
 [-0.21511781]
 [-3.63634624]]


Let's define a new data set representing the logical "XOR" function.

In [7]:
x1 = np.array([[0,0],[0,1],[1,0],[1,1]])
y1 = np.array([[0],[1],[1],[0]])

We can define a new ANN based on the new data, train it, and look at the output.

In [8]:
ann1 = NeuralNetwork(x1,y1)
ann1.train(100)
print(ann1.output)

[[0.5       ]
 [0.50011473]
 [0.49988522]
 [0.49999995]]


Additional training will not help us out. The single layer ANN cannot describe the nonlinear XOR function 

In [9]:
ann1.train(1400)
print(ann1.output)

[[0.5]
 [0.5]
 [0.5]
 [0.5]]


## Begin Homework Portion

In [10]:
class DeepNeuralNetwork(NeuralNetwork):
    def __init__(self, x, y):
        super().__init__(x, y)

        # Set weights
        self.weights_hidden     = np.random.uniform(size=(x.shape[1], 2))
        self.bias_hidden        = np.random.uniform(size=(1, 2))
        self.weights_output     = np.random.uniform(size=(2, y.shape[1]))
        self.bias_output        = np.random.uniform(size=(1, 1))
        self.output             = np.zeros(self.y.shape)

    def train(self, i, lr):
        for _ in range (i):
            # Forward Propagation
            self.hidden_layer_input     = np.dot(self.x, self.weights_hidden) + self.bias_hidden
            self.hidden_layer_output    = self.sigmoid(self.hidden_layer_input)
            self.output_layer_input     = np.dot(self.hidden_layer_output, self.weights_output) + self.bias_output
            self.output                 = self.sigmoid(self.output_layer_input)
            
            # Backpropagation
            self.error                  = self.y - self.output
            self.d_output               = self.error * self.sigmoid_derivative(self.output)
            
            self.error_hidden_layer     = self.d_output.dot(self.weights_output.T)
            self.d_hidden_layer         = self.error_hidden_layer * self.sigmoid_derivative(self.hidden_layer_output)
            
            # Updating Weights and Biases
            self.weights_output         += self.hidden_layer_output.T.dot(self.d_output) * lr
            self.bias_output            += np.sum(self.d_output, axis=0, keepdims=True) * lr
            self.weights_hidden         += self.x.T.dot(self.d_hidden_layer) * lr
            self.bias_hidden            += np.sum(self.d_hidden_layer, axis=0, keepdims=True) * lr



In [11]:
ann2 = DeepNeuralNetwork(x1,y1)
ann2.train(50000, 0.1)
print('Final predictions:')
print(ann2.output)
print('\nFinal hidden weights and bias:')
print(ann2.weights_hidden, '\n', ann2.bias_hidden)
print('\nFinal output weights and bias:')
print(ann2.weights_output, '\n', ann2.bias_output)

Final predictions:
[[0.01925543]
 [0.98336582]
 [0.9833735 ]
 [0.01722527]]

Final hidden weights and bias:
[[4.58811077 6.46772612]
 [4.58638776 6.46067251]] 
 [[-7.03972497 -2.87152512]]

Final output weights and bias:
[[-10.28268276]
 [  9.58673232]] 
 [[-4.43517739]]
